# Sentinel 2 data

In [ ]:
# Add the parent directory to the path to make imports work
import os
import sys

module_path = os.path.abspath(os.path.join("../src"))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import rasterio
import rasterio.mask
import rasterio.plot

import plot
import prepare_data
import utils
from smooth_tiled_predictions import predict_img_with_smooth_windowing

In [ ]:
import importlib
importlib.reload(plot)
importlib.reload(prepare_data)
importlib.reload(utils)

In [ ]:
# Directory paths
dir_root = Path("..")
dir_data = Path(dir_root, "data")
dir_data_sentinel = Path(dir_data, "sentinel")
dir_source = Path(
    dir_data_sentinel,
    "source",
    "S2B_MSIL2A_20230421T101559_N0509_R065_T32UQD_20230421T131333.SAFE/GRANULE/L2A_T32UQD_A031980_20230421T101820/IMG_DATA/R10m/",
)
dir_output = Path(dir_data_sentinel, "output")
dir_preprocessed = Path(dir_data_sentinel, "preprocessed")
dir_patches = Path(dir_data_sentinel, "patches")
dir_results = Path(dir_data_sentinel, "predict", "results")
dir_models = Path(dir_data, "models")

# File paths
geojson_path = Path(dir_data, "geojson", "berlin-wedding.geojson")

band_2_path = Path(dir_source, "T32UQD_20230421T101559_B02_10m.jp2")  # Blue
band_3_path = Path(dir_source, "T32UQD_20230421T101559_B03_10m.jp2")  # Green
band_4_path = Path(dir_source, "T32UQD_20230421T101559_B04_10m.jp2")  # Red

file_model = Path(
    dir_models, f"landcover_25_epochs_resnet34_backbone_batch16_iou_0.76.hdf5"
)

In [ ]:
# Clip to Berlin
band_2_path_clipped = utils.clip_jp2_to_geojson(band_2_path, geojson_path, target_dir=dir_output)
band_3_path_clipped = utils.clip_jp2_to_geojson(band_3_path, geojson_path, target_dir=dir_output)
band_4_path_clipped = utils.clip_jp2_to_geojson(band_4_path, geojson_path, target_dir=dir_output)


In [ ]:
red = rasterio.open(band_4_path_clipped)
red_array = red.read(1)
red_array_n = utils.normalize(red_array)

green = rasterio.open(band_3_path_clipped)
green_array = green.read(1)
green_array_n = utils.normalize(green_array)

blue = rasterio.open(band_2_path_clipped)
blue_array = blue.read(1)
blue_array_n = utils.normalize(blue_array)

plot.plot_rgb_histogram(red_array_n, green_array_n, blue_array_n)

In [ ]:
# Create RGB composite file
target_file = Path(dir_output, "rgb_composite.tiff")
rgb_composite = utils.create_rgb_composite_2(
    band_4_path_clipped,
    band_3_path_clipped,
    band_2_path_clipped,
    target_file,
    driver_in="GTiff",
    alpha=0.075,
)

In [ ]:
rgb = rasterio.open(rgb_composite)
rgb_array = rgb.read()

plot.plot_rgb_histogram(rgb_array[0], rgb_array[1], rgb_array[2])

In [ ]:
# Create patches
prepare_data.create_patches(dir_preprocessed, dir_patches, patch_size=256)